In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = 9, 6
%load_ext autoreload
%autoreload 2

In [ ]:
from keras import backend as K
#K.set_learning_phase(0)
%run asas_full.py 64 2 0.25 --no_train --model_type gru --lr 1e-3 --sim_type asas_full/n200 --nb_epoch 1 --n_min 200 --n_max 200 --m_max 32 --embedding 64
# --filter_length 7 --batch_norm

# Visualizing predictions

In [ ]:
%%time
pred = model.predict({'main_input': X[:100], 'aux_input': np.delete(X[:100], 1, axis=2)}, batch_size=32)

In [ ]:
i = -1

In [ ]:
#inds = ~np.isnan(X_raw[i, :, 0])
inds = np.arange(args.n_min)
i += 1
#i = np.random.choice(inds)
t = X_raw[i, inds, 0]
m = X[i, inds, 1]
#m = np.random.normal(size=len(t))
e = X_raw[i, inds, 2]

plt.errorbar(t, m, e, None, 'o');
#plt.plot(t, pred[i], 'o');
plt.errorbar(t, pred[i], None, None, 'o');

#from gatspy.periodic import LombScargleFast
#gat_resids = np.zeros(len(X))
#model_gat = LombScargleFast(fit_period=True, optimizer_kwds={'period_range': (0.05 * (t.max() - t.min()), 0.95 * (t.max() - t.min())), 'quiet': True}, silence_warnings=True)
#model_gat.fit(t, m)
#print(model_gat.score(model_gat.best_period).item())

#w_r, A_r, phi_r, b_r = pred_gru[i]
#x_r = A_r * np.sin(2 * np.pi * w_r * t + phi_r) + b_r
#plt.plot(t, x_r, '--')

plt.legend(['Original', 'Reconstructed'])
plt.title("MSE: {}, Var: {}".format((np.mean((m - np.squeeze(pred[i])) ** 2)), np.var(m)));

# Visualizing training progress

# New data

In [ ]:
import cesium.datasets
from keras.utils.np_utils import to_categorical

data = cesium.datasets.fetch_asas_training()

# Discard samples from classes w/ insufficient examples
top_classes = ['Classical_Cepheid', 'Mira', 'RR_Lyrae_FM', 'W_Ursae_Maj']
inds = np.where(np.in1d(data['classes'], top_classes))[0]
data = {'times': [data['times'][i] for i in inds],
        'measurements': [data['measurements'][i] for i in inds],
        'errors': [data['errors'][i] for i in inds],
        'classes': [data['classes'][i] for i in inds]
       }

X_list = [np.c_[t, m, e] for t, m, e in zip(data['times'], data['measurements'], data['errors'])]

sub_array_list = [np.array_split(x, np.arange(args.n_max, len(x), step=args.n_max)) for x in X_list]
sub_array_list = [[el for el in x if len(el) >= args.n_min] for x in sub_array_list]
X_list = [el for x in sub_array_list for el in x]

classnames, indices = np.unique(data['classes'], return_inverse=True)
y = np.repeat(indices, [len(x) for x in sub_array_list])
Y = to_categorical(y, len(classnames))

X_raw = pad_sequences(X_list, value=np.nan, dtype='float', padding='post')
X, scale_params = preprocess(X_raw, args.m_max)

# Remove errors
X = X[:, :, :2]

In [ ]:
lc = LightCurve.get()

In [ ]:
encode_layer = [l for l in model.layers if l.name == 'encoding'][0]
inputs = [l for l in model.layers if 'Input' in str(l)]
encode = K.function([inputs[0].input, inputs[1].input], [encode_layer.output])
encoding, = encode([X, X[:, :, 0:1]])
encoding.shape

In [ ]:
i = -1

In [ ]:
inds = ~np.isnan(X_raw[i, :, 0])
#inds = np.arange(args.n_min)
i += 1
#i = np.random.choice(inds)
t = X_raw[i, inds, 0]
m = X[i, inds, 1]
#m = np.random.normal(size=len(t))
e = X_raw[i, inds, 2]

plt.errorbar(t, m, e, None, 'o');
#plt.plot(t, pred[i], 'o');
#plt.errorbar(t, pred[i], None, None, 'o');

from gatspy.periodic import LombScargleFast
gat_resids = np.zeros(len(X))
model_gat = LombScargleFast(fit_period=True, optimizer_kwds={'period_range': (0.001 * (t.max() - t.min()), 0.95 * (t.max() - t.min())), 'quiet': True}, silence_warnings=True)
model_gat.fit(t, m)
print("Best period: {} (Score: {})".format(model_gat.best_period, model_gat.score(model_gat.best_period).item()))

#w_r, A_r, phi_r, b_r = pred_gru[i]
#x_r = A_r * np.sin(2 * np.pi * w_r * t + phi_r) + b_r
#plt.plot(t, x_r, '--')

plt.legend(['Original'])
plt.title("i={} ({})".format(i, classnames[y[i]]))

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

enc_train, enc_test, y_train, y_test = train_test_split(encoding, y, test_size=0.25, random_state=42)

#asas_model = RandomForestClassifier(n_estimators=128)
asas_model = GridSearchCV(RandomForestClassifier(), {'n_estimators': [100, 250], 'criterion': ['gini', 'entropy'], 'max_features': [0.1, 0.2, 0.3]})
#asas_model = SVC()
asas_model.fit(enc_train, y_train)

In [ ]:
print("Training score: {}%".format(100 * asas_model.score(enc_train, y_train)))
print("Test score: {}%".format(100 * asas_model.score(enc_test, y_test)))

# Light curve database

In [13]:
import glob
import os
from io import StringIO

import pandas as pd
from gatspy.periodic import LombScargleFast

from db_models import db, LightCurve

LightCurve.delete().where(LightCurve.survey == "ASAS").execute()
data_source = []
for fname in glob.glob('./data/asas/*/*'):
    with open(fname) as f:
        dfs = [pd.read_csv(StringIO(chunk), comment='#', delim_whitespace=True) for chunk in f.read().split('#     ')[1:]]
        if len(dfs) > 0:
            df = pd.concat(dfs)[['HJD', 'MAG_0', 'MER_0', 'GRADE']].sort_values(by='HJD')
            df = df[df.GRADE <= 'B']

            data_source.append({'name': os.path.basename(fname), 'survey': 'ASAS', 'times': df.HJD.values, 
                                'measurements': df.MAG_0.values, 'errors': df.MER_0.values,
#                                'best_period': model_gat.best_period, 'best_score': model_gat.score(model_gat.best_period).item()
                               })
            
with db.atomic():
    for idx in range(0, len(data_source), 100):
        LightCurve.insert_many(data_source[idx:idx + 100]).execute()

In [ ]:
%%time
for lc in LightCurve.select().where(LightCurve.best_score >> None):
    model_gat = LombScargleFast(fit_period=True, optimizer_kwds={'period_range': (0.005 * (max(lc.times) - min(lc.times)), 0.95 * (max(lc.times) - min(lc.times))), 'quiet': True}, silence_warnings=True)
    model_gat.fit(lc.times, lc.measurements, lc.errors)
    lc.best_period = model_gat.best_period
    lc.best_score = model_gat.score(model_gat.best_period).item()
    lc.save()

In [ ]:
%%time
from db_models import LightCurve

bigmacc = pd.read_csv('data/asas/bigmacc.txt', delimiter='\t')
with db.atomic():
    for i, row in bigmacc.iterrows():
        try:
            lc = LightCurve.get(name=row.ASAS_ID)
            lc.label = row.CLASS
            lc.save()
        except LightCurve.DoesNotExist:
            pass